In [ ]:
from astropy.table import Table 
from bqplot import LinearScale, Scatter, Axis, Figure, Lines
import numpy as np
from ipywidgets import HBox

In [ ]:
anth_emit = Table.read('anthro_emissions.ecsv', format='ascii.ecsv')
ipcc_future = Table.read('ipcc_future_scenarios.csv')
gases = Table.read('ipcc-gas-concentrations.csv', format='ascii.ecsv')
temps = Table.read('graph.csv')

In [ ]:
rpc_models = {
    'RCP2.6': ipcc_future['RCP2.6'],
    'RCP4.5': ipcc_future['RCP4.5'],
    'RCP6.0': ipcc_future['RCP6.0'],
    'RCP8.5': ipcc_future['RCP8.5']
}

In [ ]:
def gtc_to_ppm(gtc):
    """
    Return amount of carbon in Gt from concetration in parts per million.
    """
    mw_carbon = 12
    mw_air = 29
    mass_air = 5.184e6 # Gt
    
    return (gtc / mw_air)/(mass_air / mw_air) * 1e6


# Key Assumptions

+ The model from [this notebook]() is adopted, with $\rho_b=$353ppm, $T_b=287.35$K, $f=0.27$ and $T_{bare}$=255K. 
+ Based on the description in the Technical Summary for Working Group 1 for the IPCC AR5, 43% of anthropogenic emissions go into the atmosphere. See section TS2.8 on p. 50 of [this document](http://www.ipcc.ch/pdf/assessment-report/ar5/wg1/WG1AR5_TS_FINAL.pdf).
+ Future emissions are integrated to update future concentration numbers, and the emission graph is presented in GtC/yr instead of ppm/yr to match what is easily available in the Summary for Policymakers.

In [ ]:

#def temp_model(concentration, temp_bare=255, f=0.25, Tb=286.6, ppm_b=280):

def temp_model(concentration, temp_bare=255, f=0.27, Tb=287.35, ppm_b=353):
    n_b = (Tb/temp_bare)**4 -1
    A = f * n_b / ppm_b**0.5
    B = f + (1 - f) * (1 + n_b)
    out_temp = (A * np.sqrt(concentration) + B)**0.25 * temp_bare
    return out_temp

In [ ]:
future_year = [anth_emit['Year'][-1]] +  [2020, 2040, 2060, 2080, 2100]
constant_emit = [anth_emit['Total'][-1]] * len(future_year)
no_emit = [0] * len(future_year)

In [ ]:
# Start by defining a scale for each axis
sc_x = LinearScale(min=1950, max=2100)

# The amplitudes are limited to ±1 for this example...
sc_y = LinearScale()  # min=0, max=30000

future_handles = Scatter(x=future_year, y=constant_emit,
                         scales={'x': sc_x, 'y': sc_y}, 
                         colors=['orange'],
                         enable_move=True)
future_line = Lines(name='interactive projection', 
                    x=future_handles.x,
                    y=future_handles.y,
                    scales={'x': sc_x, 'y': sc_y},
                    colors=['orange'])

history = Lines(name='historical', x=anth_emit['Year'], y=anth_emit['Total'],
             scales={'x': sc_x, 'y': sc_y}, 
               colors=['gray'])

rpc_lines = []
colors = ['red', 'orange', 'blue', 'green']
for k, v in rpc_models.items():
    tmp = [dat for dat in v]
    c = colors.pop()
    l = Lines(name=k, x=future_year, y=[anth_emit['Total'][-1]] + tmp,
              scales={'x': sc_x, 'y': sc_y},
              colors=[c],
              opacities=[0.2])
    rpc_lines.append(l)
    
# Only allow points to be moved vertically...
future_handles.restrict_y = True

# Define the axes themselves
ax_x = Axis(scale=sc_x, label='Year')
ax_y = Axis(scale=sc_y, orientation='vertical', label='GtC/yr', label_offset='3em')

# The graph itself...
amplitude_control = Figure(marks=[future_handles, future_line, history] + rpc_lines, axes=[ax_x, ax_y], 
                           title='CO2 emissions')
def update_line(change):
    future_line.y = change['new']

future_handles.observe(update_line, names=['y'])

In [ ]:
# Start by defining a scale for each axis
sc_x = LinearScale(min=1950, max=2100)

# The amplitudes are limited to ±1 for this example...
sc_y = LinearScale()

temp_line = Lines(name='temperature',scales={'x': sc_x, 'y': sc_y})

ax_x = Axis(scale=sc_x, label='Year')
ax_y = Axis(scale=sc_y, orientation='vertical', label='Temperature')

temperature_graph = Figure(marks=[temp_line], axes=[ax_x, ax_y], title='temperature forecast')

In [ ]:
def future_concentration():
    integrated_emit = np.cumsum(future_handles.y[1:]*(future_handles.x[1:] - future_handles.x[0:-1]))
    future = gtc_to_ppm(integrated_emit/1000) + gases['CO2'][-1]
    return future  

In [ ]:
def update_prediction(change):
    future_gases_ppm = future_concentration()
    future_years = future_handles.x[1:]
    years = np.append(gases['Year'], future_years)
    concentrations = np.append(gases['CO2'], future_gases_ppm)
    temps = temp_model(concentrations)
    temp_line.x = years
    temp_line.y = temps - 287

In [ ]:
future_handles.observe(update_prediction, names=['y'])

In [ ]:
box = HBox()
amplitude_control.layout.width = '50%'
temperature_graph.layout.width = '50%'
box.children = [amplitude_control, temperature_graph]
update_prediction(None)
box